In [1]:
import os
from glob import glob
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%load_ext iversions
%iversions

plt.rc('font', family='Arial')
THREADS = 4

    pandas  2.1.4
   seaborn  0.13.2

Python 3.9.13 [Tue Sep 24, 2024 09:22:53]


In [ ]:
def run_fastqc(in_dir, out_dir):
    os.subprocess(['fastqc',in_dir,'-o','../02-OUTPUT/01-QC/'])
    return None

def run_multiqc(in_dir, out_dir):
    os.subprocess(['multiqc',in_dir,'-o','../02-OUTPUT/01-QC/'])
    return None   

def run_trimmomatic(in_dir,out_dir,params):
    for seq in glob(f"{in_dir}/*R1*.fastq.gz"):
        handle = seq.split('/')[-1].replace("_R1_001.fastq.gz","")
        r1 = handle + "_R1_001"
        r2 = handle + "_R2_001"
        os.system(f"trimmomatic PE -threads {THREADS} -phred33 -trimlog {out_dir}/02-trimmomatic-trim_log_{handle}.txt\
        {in_dir}/{r1}.fastq.gz {in_dir}/{r2}.fastq.gz \
        {out_dir}/{r1}.trimmed.fastq.gz {out_dir}/{r1}un.trimmed.fastq.gz \
        {out_dir}/{r2}.trimmed.fastq.gz {out_dir}/{r2}un.trimmed.fastq.gz \
        {params}")
    return None     

def build_bowtie2_index(in_file, out_file):
    os.subprocess(f"bowtie2-build {in_file} {out_file}")
    return None

def run_bowtie2_alignment(in_dir, out_dir):
    for seq in glob(f"{in_dir}/*R1*.fastq.gz"):
        handle = seq.split('/')[-1].replace("_R1_001.fastq.gz","")
        r1 = handle + "_R1_001"
        r2 = handle + "_R2_001"
        os.system(f"bowtie2 -q --phred33 --no-unal \
        -x {out_dir}/Gent_RK2 \
        -1 {in_dir}/{r1}.trimmed.fastq.gz \
        -2 {in_dir}/{r2}.trimmed.fastq.gz \
        -S {out_dir}/{handle}_mapping.sam")
    return None

def run_pysamstats(in_dir,out_dir):
    for sam in glob(f"{in_dir}/*.sam"):
        handle = sam.replace('.sam','').split('/')[-1]
        os.system(f"samtools view -bS {in_dir}/{handle}.sam > {out_dir}/{handle}.bam")
        os.system(f"samtools sort {out_dir}/{handle}.bam > {out_dir}/{handle}.sorted.bam")
        os.system(f"samtools index {out_dir}/{handle}.sorted.bam")
    for bam in glob("../02-OUTPUT/04-ANALYZE/*.sorted.bam"):
        os.system(f"pysamstats \
        --fasta ../01-INPUT/03-REFERENCE/Gent_RK2.fasta \
        --type variation \
        --max-depth 10000000 \
        {bam} > {bam}.var.txt")
    return None

In [ ]:
# perform QC on sequencing library using fastqc/multiqc
run_fastqc(in_dir='../01-INPUT/02-FASTQ/*',out_dir='../02-OUTPUT/01-QC/')
run_multiqc(in_dir='../02-OUTPUT/01-QC/',out_dir='../02-OUTPUT/01-QC/')

# trim sequencing reads using trimmomatic
adapter_file = "~/anaconda3/pkgs/trimmomatic-0.39-hdfd78af_2/share/trimmomatic-0.39-2/adapters/NexteraPE-PE.fa"
run_trimmomatic(in_dir="../01-INPUT/02-FASTQ",
                out_dir="../02-OUTPUT/02-TRIM",
                params=f"ILLUMINACLIP:{adapter_file}:2:30:10 LEADING:3 TRAILING:3 SLIDINGWINDOW:4:30 MINLEN:20")

# align sequencing reads to reference using bowtie2
build_bowtie2_index(in_file="../01-INPUT/03-REFERENCE/Gent_RK2.fasta",
                    out_file="../02-OUTPUT/03-ALIGN/Gent_RK2")
run_bowtie2_alignment(in_dir="../02-OUTPUT/02-TRIM",
                      out_dir="../02-OUTPUT/03-ALIGN")

# analyze sequencing results using pysamstats
run_pysamstats(in_dir="../02-OUTPUT/03-ALIGN",
               out_dir="../02-OUTPUT/04-ANALYZE")